# Analise de popularidade dos Artistas - Spotify

## Libs

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials #To access authorised Spotify data
import time 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
#import shap

#jmcabreira/A-Music-Taste-Analysis-Using-Spotify-API-and-Python.

In [3]:
#!pip install shap

## Extração de dados Spotify

In [20]:
def albumSongs(uri): 

    album = uri #assign album uri to a_name 
    spotify_albums[album] = {} #Creates dictionary for that specific album 
    #Create keys-values of empty lists inside nested dictionary for album 
    spotify_albums[album]['album'] = [] #create empty list 
    spotify_albums[album]['track_number'] = [] 
    spotify_albums[album]['id'] = [] 
    spotify_albums[album]['name'] = [] 
    spotify_albums[album]['uri'] = [] 
    global tracks
    tracks = sp.album_tracks(album) #pull data on album tracks 
    
    
    for n in range(len(tracks['items'])): #for each song track 
        spotify_albums[album]['album'].append(album_names[album_count]) #append album name tracked via album_count 
        spotify_albums[album]['track_number'].append(tracks['items'][n]['track_number']) 
        spotify_albums[album]['id'].append(tracks['items'][n]['id']) 
        spotify_albums[album]['name'].append(tracks['items'][n]['name']) 
        spotify_albums[album]['uri'].append(tracks['items'][n]['uri'])

    spotify_albums[album]['acousticness'] = [] 
    spotify_albums[album]['danceability'] = [] 
    spotify_albums[album]['energy'] = [] 
    spotify_albums[album]['instrumentalness'] = [] 
    spotify_albums[album]['liveness'] = [] 
    spotify_albums[album]['loudness'] = [] 
    spotify_albums[album]['speechiness'] = [] 
    spotify_albums[album]['tempo'] = [] 
    spotify_albums[album]['valence'] = [] 
    spotify_albums[album]['popularity'] = [] 
    track_count = 0 
    for track in spotify_albums[album]['uri']: 
        global features
        #pull audio features per track  
        features = sp.audio_features(track) 

        #Append to relevant key-value 
        spotify_albums[album]['acousticness'].append(features[0]['acousticness']) 
        spotify_albums[album]['danceability'].append(features[0]['danceability']) 
        spotify_albums[album]['energy'].append(features[0]['energy']) 
        spotify_albums[album]['instrumentalness'].append(features[0]['instrumentalness']) 
        spotify_albums[album]['liveness'].append(features[0]['liveness']) 
        spotify_albums[album]['loudness'].append(features[0]['loudness']) 
        spotify_albums[album]['speechiness'].append(features[0]['speechiness']) 
        spotify_albums[album]['tempo'].append(features[0]['tempo']) 
        spotify_albums[album]['valence'].append(features[0]['valence']) 
        #popularity is stored elsewhere 
        pop = sp.track(track) 
        spotify_albums[album]['popularity'].append(pop['popularity']) 
        track_count+=1

In [21]:
def get_dict_df(spotify_albums_aux):
    dic_df = {} 
    dic_df['album'] = [] 
    dic_df['track_number'] = [] 
    dic_df['id'] = [] 
    dic_df['name'] = [] 
    dic_df['uri'] = [] 
    dic_df['acousticness'] = [] 
    dic_df['danceability'] = [] 
    dic_df['energy'] = [] 
    dic_df['instrumentalness'] = [] 
    dic_df['liveness'] = [] 
    dic_df['loudness'] = [] 
    dic_df['speechiness'] = [] 
    dic_df['tempo'] = [] 
    dic_df['valence'] = [] 
    dic_df['popularity'] = [] 
    
    for i in spotify_albums_aux: 
        for j in spotify_albums_aux[i]: 
            dic_df[j].extend(spotify_albums_aux[i][j]) 
            len(dic_df['album'])
    
    df = pd.DataFrame.from_dict(dic_df) 
    df['artist'] = artist 
    return(df)

In [22]:
def get_songs_features(client_id,client_secret,artist):

    global album_count
    global spotify_albums
    global album_names
    global album_uris
    global sp
    
    client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret) 
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) #spotify object to access API 

    result = sp.search(artist) #search query 

    #Extract Artist's uri 
    artist_uri = result['tracks']['items'][0]['artists'][0]['uri'] 
    sp_albums = sp.artist_albums(artist_uri, album_type='album') #Store artist's albums' names' and uris in separate lists 
    album_names = [i['name'] for i in sp_albums['items']]
    album_uris = [i['uri'] for i in sp_albums['items']]
    
    global album_count
    global spotify_albums
    
    spotify_albums = {} 
    album_count = 0 
    sleep_min = 2 
    sleep_max = 5 
    start_time = time.time() 
    request_count = 0 
    
    for i in album_uris: #each album 
        albumSongs(i) 
        print("Album " + str(album_names[album_count]) + " songs has been added to spotify_albums dictionary") 
        album_count+=1 #Updates album count once all tracks have been added
        request_count+=1 
        if request_count % 5 == 0: 
            print(str(request_count) + " playlists completed") 
            time.sleep(np.random.uniform(sleep_min, sleep_max)) 
            print('Loop #: {}'.format(request_count)) 
            print('Elapsed Time: {} seconds'.format(time.time() - start_time))
    dados=get_dict_df(spotify_albums)        
    return(dados)

## Extração um a um

In [45]:
#Variaveis entrada
client_id= "eadfbc70b4814c72a920301b436ebcbf" 
client_secret= "2aa7d68f735d48ccbe3221e09cdd8e7d" 
artist = "Djavan" #chosen artist 
dados=get_songs_features(client_id,client_secret,artist)

Album Dior Deposits (Deluxe) songs has been added to spotify_albums dictionary
Album Scamboy Color songs has been added to spotify_albums dictionary


In [ ]:
dados_final=pd.concat([dados_final,dados])
dados_final.to_csv('dados_musicas.csv', index = False)

## Extração em massa

In [46]:
artistas=["jack johnson","o rappa","tiago iorc","duda beat","anitta",
          "pablo vittar","Rolling Stones","Marcelo Camelo"]
artistas

['jack johnson',
 'o rappa',
 'tiago iorc',
 'duda beat',
 'anitta',
 'pablo vittar',
 'Rolling Stones',
 'Marcelo Camelo']

In [47]:
for i in artistas:
    print(i)
    #Variaveis entrada
    client_id= "eadfbc70b4814c72a920301b436ebcbf" 
    client_secret= "2aa7d68f735d48ccbe3221e09cdd8e7d" 
    artist = i #chosen artist 
    dados=get_songs_features(client_id,client_secret,artist)
    dados_final=pd.concat([dados_final,dados])


jack johnson
Album All The Light Above It Too songs has been added to spotify_albums dictionary
Album From Here To Now To You songs has been added to spotify_albums dictionary
Album From Here To Now To You songs has been added to spotify_albums dictionary
Album From Here To Now To You songs has been added to spotify_albums dictionary
Album From Here To Now To You (Commentary Version) songs has been added to spotify_albums dictionary
5 playlists completed
Loop #: 5
Elapsed Time: 29.872482776641846 seconds
Album Brushfire Fairytales [Remastered (Bonus Version)] songs has been added to spotify_albums dictionary
Album To The Sea songs has been added to spotify_albums dictionary
Album To The Sea songs has been added to spotify_albums dictionary
Album To The Sea (Japan Version) songs has been added to spotify_albums dictionary
Album To The Sea (Telus Exclusive) songs has been added to spotify_albums dictionary
10 playlists completed
Loop #: 10
Elapsed Time: 64.66368246078491 seconds
Album En

In [48]:
dados_final.to_csv('dados_musicas.csv', index = False)


## Leitura dos dados CSV

In [26]:
dados_final=pd.read_csv("dados_musicas.csv")

## Função padroniza variável


In [3]:
def padroniza_variavel(dados,var):

    media=np.mean(dados[var])
    dp=np.std(dados[var])
    var_pad=(dados[var]-media)/dp
    var_pad[var_pad.isna()]=0
    nome_novo="std_"+var
    dados[nome_novo]=var_pad
    return("feito")

## Artistas disponíveis

In [43]:
dados_final["artist"].unique()

array(['Bruno Mars', 'Djavan', 'Jonh Mayer', 'Ed Sheeran',
       'Marilia Mendonça', 'Caetano Veloso', 'The Killers', 'The Beatles',
       'Queen', 'Legião Urbana', '5 a seco', 'Maria Gadu', 'Marisa Monte',
       'Racionais MCs', 'Criolo', 'Djonga', 'ExaltaSamba',
       'Turma do pagode', 'Tim Maia', 'Nirvana', 'red hot chili peppers',
       'Nando reis', 'AnaVitoria', 'Chitãozinho & Xororó', 'Maneva',
       'Natiruts'], dtype=object)

## Análise exploratória

In [4]:
#Selecione 2 artistas

artista1="Queen"
artista2="The Beatles"

In [5]:
#Filtro nos dados
dados_aux=dados_final[dados_final["artist"].isin([artista1,artista2])]

### Distribuição de popularidade - por Artista

In [49]:
x1 = dados_aux[dados_aux["artist"]==artista1]["popularity"]
x2 = dados_aux[dados_aux["artist"]==artista2]["popularity"]


hist_data = [x1, x2]

group_labels = [artista1,artista2]

# Create distplot with curve_type set to 'normal'
fig = ff.create_distplot(hist_data, group_labels, #colors=colors,
                         bin_size=4, show_rug=False,show_hist=True,colors=["lightgreen","black"]
)

# Add title
fig.update_layout(title_text='Distribuição de popularidade da músicas')
fig.show()

### Distribuição de popularidade - por Album

In [50]:
dados_artista = dados_aux[dados_aux["artist"]==artista1][["popularity","album"]]
albums=dados_artista["album"].unique()
dic={}
for album in albums:
    dic[album]=dados_artista[dados_artista["album"]==album]["popularity"]

l=list(dic.values())
#group_labels = [artista1,artista2]

# Create distplot with curve_type set to 'normal'
fig = ff.create_distplot(hist_data=l, group_labels=albums,show_rug=False,bin_size=5,show_hist=False)


# Add title
fig.update_layout(title_text='Distribuição de popularidade da músicas')
fig.show()


### Correlação variáveis

In [51]:
variaveis=['acousticness',
       'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness',
       'speechiness', 'tempo', 'valence', 'popularity']

c1 = dados_aux[dados_aux["artist"]==artista1][variaveis].corr()
c2 = dados_aux[dados_aux["artist"]==artista2][variaveis].corr()

fig = go.Figure(data=go.Heatmap(
        z=c1,
        x=c1.columns,
        y=c1.index,
        colorscale=["green","white","black"]))

fig.update_layout(
    title='Correlação entre as variáveis '+artista1,
    xaxis_nticks=36)

fig.show()

fig = go.Figure(data=go.Heatmap(
        z=c2,
        x=c2.columns,
        y=c2.index,
        colorscale=["green","white","black"]))

fig.update_layout(
    title='Correlação entre as variáveis '+artista2,
    xaxis_nticks=36)

fig.show()

### Gráfico Radar

In [52]:
padroniza_variavel(dados_aux,'acousticness')
padroniza_variavel(dados_aux,'danceability')
padroniza_variavel(dados_aux,'energy')
padroniza_variavel(dados_aux,'instrumentalness')
padroniza_variavel(dados_aux,'liveness')
padroniza_variavel(dados_aux,'loudness')
padroniza_variavel(dados_aux,'speechiness')
padroniza_variavel(dados_aux,'tempo')
padroniza_variavel(dados_aux,'valence')
padroniza_variavel(dados_aux,'popularity')


aux=dados_aux.groupby("artist",as_index=False).agg({'std_danceability':np.mean,
 'std_energy':np.mean,
 'std_instrumentalness':np.mean,
 'std_popularity':np.mean})

v1=np.asarray(aux[aux["artist"]==artista1].iloc[0,1:])
v1=np.concatenate((v1,[v1[0]]))
v2=np.asarray(aux[aux["artist"]==artista2].iloc[0,1:])
v2=np.concatenate((v2,[v2[0]]))

variaveis=["danceability","energy","instrumentalness","popularity"]
fig = go.Figure()

fig.add_trace(go.Scatterpolar(
      r=v1,
      theta=variaveis,
      fill='toself',
      name=artista1,
    line =  dict(
            color = 'green'
        )
    
))
fig.add_trace(go.Scatterpolar(
      r=v2,
      theta=variaveis,
      fill='toself',
      name=artista2,
        line =  dict(
            color = 'gray'
        )
))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[-1, 1]
    )),
  showlegend=True
)

fig.show()

## Construção de Modelo

In [57]:
#Libs modelo
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import confusion_matrix

In [58]:
variaveis=['acousticness',
       'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness',
       'speechiness', 'tempo', 'valence', 'popularity']

variaveis_std=['std_'+i for i in variaveis]

X=dados_aux[variaveis_std]
y=dados_aux["artist"]

In [59]:
rf=RandomForestClassifier(max_depth=10,n_estimators=30,random_state=0)
rf_scores = cross_validate(rf, X, y,
                        scoring='roc_auc', cv=5,
                         return_estimator=True)
gboost=xgb.XGBClassifier(
                        n_estimators=30,
                        max_depth=10
                            )
gboost_scores = cross_validate(gboost, X, y,
                        scoring='roc_auc', cv=5,
                         return_estimator=True)

## Seleção do modelo

In [68]:
rf_auc=rf_scores["test_score"].mean()
gboost_auc=gboost_scores["test_score"].mean()

escolha=max([rf_auc,gboost_auc])
if escolha==rf_auc:
    print("Modelo escolhido: Random Forest (AUC:{})".format(rf_auc.round(2)))
elif escolha==gboost_auc:
    print("Modelo escolhido: XGBoost (AUC:{})".format(gboost_auc.round(2)))  

Modelo escolhido: Random Forest (AUC:0.98)


## Metricas modelo

In [63]:
#Randon forest
y_pred=rf.fit(X_train,y_train).predict(X_test)
confusion_matrix(y_test, y_pred, labels=[artista1,artista2])

array([[ 23,  11],
       [  1, 151]])

In [64]:
#XGBoost
y_pred=gboost.fit(X_train,y_train).predict(X_test)
confusion_matrix(y_test, y_pred, labels=[artista1,artista2])

array([[ 24,  10],
       [  1, 151]])

## Importancia das variáveis

In [70]:
#Randon Forest
f_imp=rf.fit(X_train,y_train).feature_importances_
aux={"variaveis":variaveis,"importancia":f_imp}
aux1=pd.DataFrame(aux)
aux2=aux1.sort_values(by="importancia",ascending=True)
fig = go.Figure(go.Bar(y=aux2["variaveis"],x=aux2["importancia"],orientation='h'))
fig.show()
rf.feature_importances_
aux1.sort_values(by="importancia",ascending=False)

,variaveis,importancia
1,danceability,0.185501
5,loudness,0.168635
6,speechiness,0.125925
8,valence,0.102696
3,instrumentalness,0.100052
0,acousticness,0.098717
9,popularity,0.070972
2,energy,0.061707
4,liveness,0.046245
7,tempo,0.039549


In [71]:
#XGBoost
f_imp=gboost.fit(X_train,y_train).feature_importances_
aux={"variaveis":variaveis,"importancia":f_imp}
aux1=pd.DataFrame(aux)
aux2=aux1.sort_values(by="importancia",ascending=True)
fig = go.Figure(go.Bar(y=aux2["variaveis"],x=aux2["importancia"],orientation='h'))
fig.show()

,variaveis,importancia
1,danceability,0.204504
3,instrumentalness,0.195842
5,loudness,0.177318
0,acousticness,0.122082
6,speechiness,0.077520
2,energy,0.066133
8,valence,0.056078
4,liveness,0.041290
9,popularity,0.031003
7,tempo,0.028229


## Predição por música

In [77]:
escolha=max([rf_auc,gboost_auc])
if escolha==rf_auc:
    y_pred=rf.fit(X_train,y_train).predict(X)
    y_prob=rf.fit(X_train,y_train).predict_proba(X)
        
        
elif escolha==gboost_auc:
    y_pred=gboost.fit(X_train,y_train).predict(X)
    y_prob=gboost.fit(X_train,y_train).predict_proba(X)
#Tabela

    
df_aux=pd.DataFrame({"Musica":dados_aux["name"],
                         "Artista":dados_aux["artist"],
                         "Artista Modelo":y_pred,
                         "Score Artista1":y_prob[:,0],
                         "Score Artista2":y_prob[:,1],
                         'std_acousticness':dados_aux["std_acousticness"],
                         'std_danceability':dados_aux["std_danceability"],
                         'std_energy':dados_aux["std_energy"],
                         'std_instrumentalness':dados_aux["std_instrumentalness"],
                         'std_liveness':dados_aux["std_liveness"],
                         'std_loudness':dados_aux["std_loudness"],
                         'std_speechiness':dados_aux["std_speechiness"],
                         'std_tempo':dados_aux["std_tempo"],
                         'std_valence':dados_aux["std_valence"],
                         'std_popularity':dados_aux["std_popularity"]})
df_aux

,Musica,Artista,Artista Modelo,Score Artista1,Score Artista2,std_acousticness,std_danceability,std_energy,std_instrumentalness,std_liveness,std_loudness,std_speechiness,std_tempo,std_valence,std_popularity
1488,Come Together - 2019 Mix,The Beatles,The Beatles,0.070000,0.930000,-0.879801,-0.144390,-0.779832,0.626715,-0.680510,-0.232021,-0.349208,1.605693,-1.684756,0.906787
1489,Something - 2019 Mix,The Beatles,The Beatles,0.010063,0.989937,-0.836089,-0.868100,-0.655259,-0.316571,-0.070362,0.189072,-0.422006,0.498391,-0.468425,0.795110
1490,Maxwell's Silver Hammer - 2019 Mix,The Beatles,The Beatles,0.060000,0.940000,0.462956,1.544268,-0.650276,-0.316075,0.314544,0.174460,-0.363881,0.421319,0.509332,0.683433
1491,Oh! Darling - 2019 Mix,The Beatles,The Beatles,0.000063,0.999937,-1.034896,-0.741451,0.759890,-0.294289,0.495117,0.949962,-0.356545,1.913388,-0.249408,0.795110
1492,Octopus's Garden - 2019 Mix,The Beatles,The Beatles,0.020063,0.979937,-0.780397,0.030507,0.640300,-0.313070,-0.564563,1.037634,-0.392097,-0.940193,0.188627,0.683433
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2101,Idfc,Queen,The Beatles,0.181429,0.818571,0.832077,0.422517,-0.082223,-0.316372,-0.612082,0.357243,-0.234651,0.904013,-1.027703,2.246909
2102,Waste Away (feat. Devon Baldwin),Queen,Queen,0.650063,0.349937,0.042029,-0.295163,-0.585498,-0.316601,-0.616834,0.013727,-0.054632,0.628949,-0.718731,0.627595
2103,Dirty Laundry,Queen,Queen,0.780000,0.220000,-0.981147,1.242722,0.615386,-0.316601,-0.027595,0.318986,-0.384761,-0.667127,0.360713,1.241817
2104,My Heart Is Lost,Queen,The Beatles,0.250063,0.749937,-0.365946,0.500919,0.022418,-0.316601,-0.728029,0.566860,-0.288262,1.433007,0.392001,0.404241
